In [ ]:
#|default_exp optimizer.torchscript

# TorchScript Optimizers
> fastai optimizers compiled with TorchScript for improved performance

In [ ]:
#|export
from __future__ import annotations
from typing import Optional, Dict

from torch.nn import Parameter

from fastcore.basics import range_of, merge

from fastai.optimizer import (Optimizer, _update, weight_decay, l2_reg, average_grad, sgd_step, 
                              momentum_step, average_sqr_grad, rms_prop_step, step_stat, adam_step, 
                              radam_step, qhadam_step, larc_layer_lr, larc_step, lamb_step, Lookahead)

from fastxtend.imports import *

In [ ]:
#|hide
from fastxtend.test_utils import *

## Test Utils -

In [ ]:
#|hide
def tst_param(val, grad=None):
    "Create a tensor with `val` and a gradient of `grad` for testing"
    res = tensor([val]).float()
    res.grad = tensor([val/10 if grad is None else grad]).float()
    return res

In [ ]:
#|hide
def tst_params():
    r = L.range(4)
    return r.map(tst_param)

## JitOptimizer -

In [ ]:
#|exporti
def _update(
    state:dict,
    new=None # New values to update `state` dict
):
    if isinstance(new, dict): state.update(new)

In [ ]:
#|export
class JitOptimizer(Optimizer):
    @torch.no_grad()
    def step(self, closure=None):
        if closure is not None: raise NotImplementedError("fastai optimizers currently do not support closure")
        for pg, hyper in zip(self.param_lists, self.hypers):
            for p in pg:
                if hasattr(p, 'grad') and p.grad is not None:
                    _update(self.state[p], self.cbs[0](p, p.grad, **{**self.state[p], **hyper}))

## SGD -

In [ ]:
#|exporti
@torch.jit.script
def sgd_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, decouple_wd:bool, grad_avg:Optional[Tensor]=None, 
                 do_wd:bool=True, dampening:bool=False, force_train:Optional[bool]=None):
    dp = p
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if mom != 0:
        if grad_avg is None: 
            grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)

        # average_grad
        damp = 1-mom if dampening else 1.
        grad_avg = grad_avg.mul(mom)
        grad_avg = grad_avg.add(grad, alpha=damp)

        # momentum_step
        dp = dp.add(grad_avg, alpha=-lr)
        p.set_(dp)
        return {'grad_avg': grad_avg}
    else:
        # sgd_step
        dp = dp.add(grad, alpha=-lr)
        p.set_(dp)
        return None

In [ ]:
#|export
def SGD(params, lr, mom=0., wd=0., decouple_wd=True, jit=False):
    "A `Optimizer` for SGD with `lr` and `mom` and `params`"
    if jit:
        cb = partial(sgd_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, mom=mom, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        if mom != 0: cbs.append(average_grad)
        cbs.append(sgd_step if mom==0 else momentum_step)
        return Optimizer(params, cbs, lr=lr, mom=mom, wd=wd)

In [ ]:
#|hide

# Test Vanilla SGD
params_org = tst_params()
opt_org = SGD(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = tst_params()
opt_jit = SGD(params_jit, lr=0.1, jit=True)
opt_jit.step()

test_close([p.item() for p in params_org], [i*0.99 for i in range(4)])
test_close([p.item() for p in params_org], [p.item() for p in params_jit])

opt_org.step()
opt_jit.step()
test_close([p.item() for p in params_org], [i*0.98 for i in range(4)])
test_close([p.item() for p in params_org], [p.item() for p in params_jit])

## RMSProp -

In [ ]:
#|exporti
@torch.jit.script
def rmsprop_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, sqr_mom:float, eps:float, decouple_wd:bool, 
                     grad_avg:Optional[Tensor]=None, sqr_avg:Optional[Tensor]=None, do_wd:bool=True, force_train:Optional[bool]=None):
    dp = p
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if sqr_avg is None: 
        sqr_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)

    if mom != 0:
        if grad_avg is None: 
            grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)

        # average_grad, dampening=False
        grad_avg = torch.mul(grad_avg, mom)
        grad_avg = torch.add(grad_avg, grad)

        # average_sqr_grad
        sqr_avg = torch.mul(sqr_avg, sqr_mom)
        sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)

        # rms_prop_step
        denom = torch.sqrt(sqr_avg)
        denom = torch.add(denom, eps)
        dp = torch.addcdiv(dp, grad_avg, denom, value=-lr)
        p.set_(dp)
        return {'grad_avg': grad_avg, 'sqr_avg': sqr_avg}
    else:
        # average_sqr_grad
        sqr_avg = torch.mul(sqr_avg, sqr_mom)
        sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)
        
        # rms_prop_step
        denom = torch.sqrt(sqr_avg)
        denom = torch.add(denom, eps)
        dp = dp.addcdiv(grad, denom, value=-lr)
        p.set_(dp)
        return {'sqr_avg': sqr_avg}

In [ ]:
#|export
def RMSProp(params, lr, sqr_mom=0.99, mom=0., eps=1e-8, wd=0., decouple_wd=True, jit=False):
    "A `Optimizer` for RMSProp with `lr`, `sqr_mom`, `mom` and `params`"
    if jit:
        cb = partial(rmsprop_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        cbs += ([average_sqr_grad] if mom==0. else [average_grad, average_sqr_grad])
        cbs.append(rms_prop_step)
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, wd=wd, eps=eps)

In [ ]:
#|hide
#Without momentum
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = RMSProp(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = RMSProp(params_jit, lr=0.1, jit=True)
opt_jit.step()

test_close(params_org[0], tensor([0.,1.,2.]))
test_close(params_org[0], params_jit[0])

opt_org.step()
opt_jit.step()
step = - 0.1 * 0.1 / (math.sqrt((0.01*0.99+0.01) * 0.1**2) + 1e-8)
test_close(params_org[0], tensor([step, 1+step, 2+step]))
test_close(params_org[0], params_jit[0])

In [ ]:
#|hide
#With momentum
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = RMSProp(params_org, lr=0.1, mom=0.9, jit=False)
opt_org.step()

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = RMSProp(params_jit, lr=0.1, mom=0.9, jit=True)
opt_jit.step()

test_close(params_org[0], tensor([0.,1.,2.]))
test_close(params_org[0], params_jit[0])

opt_org.step()
opt_jit.step()
step = - 0.1 * (0.1 + 0.9*0.1) / (math.sqrt((0.01*0.99+0.01) * 0.1**2) + 1e-8)
test_close(params_org[0], tensor([step, 1+step, 2+step]))
test_close(params_org[0], params_jit[0])

## Adam -

In [ ]:
#|exporti
def debias(beta:float, step:int):
    "Simple debias calculation"
    return 1-beta**step

In [ ]:
#|exporti
@torch.jit.script
def adam_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, sqr_mom:float, eps:float, 
                  decouple_wd:bool, grad_avg:Optional[Tensor]=None, sqr_avg:Optional[Tensor]=None, 
                  do_wd:bool=True, step:int=0, force_train:Optional[bool]=None):
    dp = p
    step += 1
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if grad_avg is None: 
        grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if sqr_avg is None: 
        sqr_avg  = torch.zeros_like(dp, memory_format=torch.preserve_format)

    # average_grad, dampening=True
    grad_avg = torch.mul(grad_avg, mom)
    grad_avg = torch.add(grad_avg, grad, alpha=1-mom)

    # average_sqr_grad
    sqr_avg = torch.mul(sqr_avg, sqr_mom)
    sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)

    # adam_step
    debias1 = debias(mom, step)
    debias2 = debias(sqr_mom, step)
    dp = torch.addcdiv(dp, grad_avg, torch.sqrt(sqr_avg/debias2) + eps, value = -lr / debias1)
    p.set_(dp)

    return torch.jit.annotate(Dict[str, Union[Tensor, int]], {'grad_avg': grad_avg, 'sqr_avg': sqr_avg, 'step': step})

In [ ]:
#|export
def Adam(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0.01, decouple_wd=True, jit=False):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    if jit:
        cb = partial(adam_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, adam_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [ ]:
#|hide
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = Adam(params_org, lr=0.1, wd=0, jit=False)
opt_org.step()

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = Adam(params_jit, lr=0.1, wd=0, jit=True)
opt_jit.step()

step = -0.1 * 0.1 / (math.sqrt(0.1**2) + 1e-8)
test_close(params_org[0], tensor([1+step, 2+step, 3+step]))
test_close([p[0].item() for p in params_org], [p[0].item() for p in params_jit])

opt_org.step()
opt_jit.step()
test_close(params_org[0], tensor([1+2*step, 2+2*step, 3+2*step]), eps=1e-3)
test_close([p[0].item() for p in params_org], [p[0].item() for p in params_jit])

## RAdam -

In [ ]:
#|exporti
@torch.jit.script
def radam_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, sqr_mom:float, eps:float, beta:float,
                  decouple_wd:bool, grad_avg:Optional[Tensor]=None, sqr_avg:Optional[Tensor]=None, 
                  do_wd:bool=True, step:int=0, force_train:Optional[bool]=None):
    dp = p
    step += 1
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if grad_avg is None: 
        grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if sqr_avg is None: 
        sqr_avg  = torch.zeros_like(dp, memory_format=torch.preserve_format)

    # average_grad, dampening=True
    grad_avg = torch.mul(grad_avg, mom)
    grad_avg = torch.add(grad_avg, grad, alpha=1-mom)

    # average_sqr_grad
    sqr_avg = torch.mul(sqr_avg, sqr_mom)
    sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)

    # radam_step
    debias1 = debias(mom, step)
    debias2 = debias(sqr_mom, step)
    r_inf = 2/(1-sqr_mom) - 1
    r = r_inf - 2*step*sqr_mom**step/(1-sqr_mom**step)
    
    if r > 5:
        v = math.sqrt(((r-4) * (r-2) * r_inf)/((r_inf-4)*(r_inf-2)*r))
        denom = torch.sqrt(sqr_avg/debias2)
        if eps != 0: 
            denom = denom + eps
        if beta != 0: 
            denom = F.softplus(denom, beta)
        dp = torch.addcdiv(dp, grad_avg, denom, value = -lr*v / debias1)
    else:
        dp = torch.add(dp, grad_avg, alpha=-lr / debias1)
    p.set_(dp)

    return torch.jit.annotate(Dict[str, Union[Tensor, int]], {'grad_avg': grad_avg, 'sqr_avg': sqr_avg, 'step': step})

In [ ]:
#|export
def RAdam(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0., beta=0., decouple_wd=True, jit=False):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    if jit:
        cb = partial(radam_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd, beta=beta)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, radam_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd, beta=beta)

In [ ]:
#|hide
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = RAdam(params_org, lr=0.1, jit=False)

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = RAdam(params_jit, lr=0.1, jit=True)

#The r factor is lower than 5 during the first 5 steps so updates use the average of gradients (all the same)
r_inf = 2/(1-0.99) - 1
for i in range(5): 
    r = r_inf - 2*(i+1)*0.99**(i+1)/(1-0.99**(i+1))
    assert r <= 5
    opt_org.step()
    opt_jit.step()
p = tensor([0.95, 1.9, 2.85])
test_close(params_org[0], p)
test_close(params_org[0], params_jit[0])

#The r factor is greater than 5 for the sixth step so we update with RAdam
r = r_inf - 2*6*0.99**6/(1-0.99**6)
assert r > 5
opt_org.step()
opt_jit.step()
v = math.sqrt(((r-4) * (r-2) * r_inf)/((r_inf-4)*(r_inf-2)*r))
step = -0.1*0.1*v/(math.sqrt(0.1**2) + 1e-8)
test_close(params_org[0], p+step)
test_close(params_org[0], params_jit[0])

## QHAdam -

In [ ]:
#|exporti
@torch.jit.script
def qhadam_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, sqr_mom:float, eps:float,
                    nu_1:float, nu_2:float, decouple_wd:bool, grad_avg:Optional[Tensor]=None, 
                    sqr_avg:Optional[Tensor]=None, do_wd:bool=True, step:int=0, force_train:Optional[bool]=None):
    dp = p
    step += 1
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if grad_avg is None: 
        grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if sqr_avg is None: 
        sqr_avg  = torch.zeros_like(dp, memory_format=torch.preserve_format)

    # average_grad, dampening=True
    grad_avg = torch.mul(grad_avg, mom)
    grad_avg = torch.add(grad_avg, grad, alpha=1-mom)

    # average_sqr_grad
    sqr_avg = torch.mul(sqr_avg, sqr_mom)
    sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)

    # qhadam_step
    debias1 = debias(mom, step)
    debias2 = debias(sqr_mom, step)
    dp = torch.addcdiv(dp, ((1-nu_1) * grad) + (nu_1 * (grad_avg / debias1)),
                       torch.sqrt(((1 - nu_2) * (grad)**2) + (nu_2 * (sqr_avg / debias2))) + eps,
                       value = -lr)
    p.set_(dp)

    return torch.jit.annotate(Dict[str, Union[Tensor, int]], {'grad_avg': grad_avg, 'sqr_avg': sqr_avg, 'step': step})

In [ ]:
#|export
def QHAdam(params, lr, mom=0.999, sqr_mom=0.999, nu_1=0.7, nu_2=1.0, eps=1e-8, wd=0., decouple_wd=True, jit=True):
    "An `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `nus`, eps` and `params`"
    if jit:
        cb = partial(qhadam_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, nu_1=nu_1, nu_2=nu_2, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, qhadam_step]
        return Optimizer(params, cbs, lr=lr, nu_1=nu_1, nu_2=nu_2, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [ ]:
#hide
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = QHAdam(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = QHAdam(params_jit, lr=0.1, jit=True)
opt_jit.step()

step = -0.1 * (((1-0.7) * 0.1) + (0.7 * 0.1)) / (
     math.sqrt(((1-1.0) * 0.1**2) + (1.0 * 0.1**2)) + 1e-8) 

test_close(params_org[0], tensor([1+step, 2+step, 3+step]))
test_close(params_org[0], params_jit[0])

opt_org.step()
opt_jit.step()
test_close(params_org[0], tensor([1+2*step, 2+2*step, 3+2*step]), eps=1e-3)
test_close(params_org[0], params_jit[0])

## LARS/LARC -

In [ ]:
#|exporti
@torch.jit.script
def larc_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, eps:float, trust_coeff:float, decouple_wd:bool, 
                 clip:bool, grad_avg:Optional[Tensor]=None, do_wd:bool=True, dampening:bool=False, force_train:Optional[bool]=None):
    dp = p
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    # larc_layer_lr
    p_norm = torch.norm(dp)
    g_norm = torch.norm(grad)
    local_lr = lr*trust_coeff * (p_norm) / (g_norm + p_norm * wd + eps)
    if clip:
        lr = min(local_lr, lr)
    else:
        lr = local_lr

    if mom != 0:
        if grad_avg is None: 
            grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)

        # average_grad, dampening=True
        grad_avg = torch.mul(grad_avg, mom)
        grad_avg = torch.add(grad_avg, grad)

        # larc_step
        dp = torch.add(dp, grad_avg, alpha=-lr)
    else:
        # larc_step
        dp = torch.add(dp, grad, alpha=-lr)

    p.set_(dp)
    return {'grad_avg': grad_avg}

In [ ]:
#|export
def Larc(params, lr, mom=0.9, clip=True, trust_coeff=0.02, eps=1e-8, wd=0., decouple_wd=True, jit=False):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    if jit:
        cb = partial(larc_jit_step, decouple_wd=decouple_wd, clip=clip)
        return JitOptimizer(params, cb, lr=lr, mom=mom, trust_coeff=trust_coeff, eps=eps, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        if mom!=0.: cbs.append(average_grad)
        cbs += [partial(larc_layer_lr, clip=clip), larc_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, trust_coeff=trust_coeff, eps=eps, wd=wd)

In [ ]:
#hide
params_org = [tst_param([1,2,3], [0.1,0.2,0.3]), tst_param([1,2,3], [0.01,0.02,0.03])]
opt_org = Larc(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = [tst_param([1,2,3], [0.1,0.2,0.3]), tst_param([1,2,3], [0.01,0.02,0.03])]
opt_jit = Larc(params_jit, lr=0.1, jit=True)
opt_jit.step()

#First param local lr is 0.02 < lr so it's not clipped
test_close(opt_org.state[params_org[0]]['local_lr'], 0.02)
#Second param local lr is 0.2 > lr so it's clipped
test_eq(opt_org.state[params_org[1]]['local_lr'], 0.1)

test_close(params_org[0], tensor([0.998,1.996,2.994]))
test_close(params_org[0], params_jit[0])
test_close(params_org[1], tensor([0.999,1.998,2.997]))
test_close(params_org[1], params_jit[1])

In [ ]:
#hide
params_org = [tst_param([1,2,3], [0.1,0.2,0.3]), tst_param([1,2,3], [0.01,0.02,0.03])]
opt_org = Larc(params_org, lr=0.1, clip=False, jit=False)
opt_org.step()

params_jit = [tst_param([1,2,3], [0.1,0.2,0.3]), tst_param([1,2,3], [0.01,0.02,0.03])]
opt_jit = Larc(params_jit, lr=0.1, clip=False, jit=True)
opt_jit.step()

#No clipping
test_close(opt_org.state[params_org[0]]['local_lr'], 0.02)
test_close(opt_org.state[params_org[1]]['local_lr'], 0.2)
test_close(params_org[0], tensor([0.998,1.996,2.994]))
test_close(params_org[0], params_jit[0])
test_close(params_org[1], tensor([0.998,1.996,2.994]))
test_close(params_org[1], params_jit[1])

## LAMB -

In [ ]:
#|exporti
@torch.jit.script
def lamb_jit_step(p:Tensor, grad:Tensor, lr:float, wd:float, mom:float, sqr_mom:float, eps:float, 
                  decouple_wd:bool, grad_avg:Optional[Tensor]=None, sqr_avg:Optional[Tensor]=None, 
                  do_wd:bool=True, step:int=0, force_train:Optional[bool]=None):
    dp = p
    step += 1
    if do_wd and wd != 0:
        if decouple_wd:
            # weight_decay
            dp = dp.mul(1 - lr*wd)
        else:
            # l2_reg
            grad = grad.add(dp, alpha=wd)

    if grad_avg is None: 
        grad_avg = torch.zeros_like(dp, memory_format=torch.preserve_format)
    if sqr_avg is None: 
        sqr_avg  = torch.zeros_like(dp, memory_format=torch.preserve_format)

    # average_grad, dampening=True
    grad_avg = torch.mul(grad_avg, mom)
    grad_avg = torch.add(grad_avg, grad, alpha=1-mom)

    # average_sqr_grad
    sqr_avg = torch.mul(sqr_avg, sqr_mom)
    sqr_avg = torch.addcmul(sqr_avg, grad, grad, value=1-sqr_mom)

    # lamb_step
    debias1 = debias(mom, step)
    debias2 = debias(sqr_mom, step)
    r1 = dp.pow(2).mean().sqrt()
    lstep = (grad_avg/debias1) / ((sqr_avg/debias2).sqrt()+eps)
    r2 = lstep.pow(2).mean().sqrt()
    if r1 == 0 or r2 == 0:
        q = 1.
    else:
        q = min(r1/r2, 10.)
    dp = torch.add(dp,lstep, alpha = -lr * q)

    p.set_(dp)

    return torch.jit.annotate(Dict[str, Union[Tensor, int]], {'grad_avg': grad_avg, 'sqr_avg': sqr_avg, 'step': step})

In [ ]:
#|export
def Lamb(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0., decouple_wd=True, jit=False):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    if jit:
        cb = partial(lamb_jit_step, decouple_wd=decouple_wd)
        return JitOptimizer(params, cb, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay] if decouple_wd else [l2_reg]
        cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, lamb_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [ ]:
#|hide
params_org = tst_param([1,2,3], [0.1,0.2,0.3])
opt_org = Lamb(params_org, lr=0.1, jit=False)
opt_org.step()

params_jit = tst_param([1,2,3], [0.1,0.2,0.3])
opt_jit = Lamb(params_jit, lr=0.1, jit=True)
opt_jit.step()

test_close(params_org[0], tensor([0.7840,1.7840,2.7840]), eps=1e-3)
test_close(params_org[0], params_jit[0])

## ranger -

In [ ]:
#|export
@delegates(RAdam)
def ranger(p, lr, mom=0.95, wd=0.01, eps=1e-6, jit=False, **kwargs):
    "Convenience method for `Lookahead` with `RAdam`"
    return Lookahead(RAdam(p, lr=lr, mom=mom, wd=wd, eps=eps, jit=jit, **kwargs))